# Prompt Engineering for Repository Summarization

This notebook is designed to help you experiment with different prompts to generate high-quality summaries for government repositories. 

You can:
1.  Load your repository data from a CSV file.
2.  Take a random sample of repositories to test.
3.  Define and test multiple summarization prompts.
4.  Generate summaries using a language model.
5.  Compare the results in a clear, side-by-side format.

## 1. Setup

First, let's import the necessary libraries and set up our configuration.

In [43]:
import pandas as pd
from IPython.display import display, HTML
import random

# --- Configuration ---
CSV_FILE_PATH = '/home/theo/projects/iai/data/tmprunid/classified_gov_repositories_batch.csv'
SAMPLE_SIZE = 10  # Number of random repositories to test

## 2. Load and Sample Data

We'll load the data from the CSV file and select a random sample of repositories to work with. This keeps the process fast and manageable for experimentation.

In [36]:
try:
    df = pd.read_csv(CSV_FILE_PATH)
    # Ensure the sample size is not larger than the number of rows in the dataframe
    sample_size = min(SAMPLE_SIZE, len(df))
    if len(df) > sample_size:
        sample_df = df.sample(n=sample_size, random_state=62) # random_state for reproducibility
    else:
        sample_df = df
    print(f"Successfully loaded {len(df)} repositories and sampled {len(sample_df)} of them.")
    display(sample_df[['name', 'readme', 'description']].head())
except FileNotFoundError:
    print(f"Error: The file was not found at {CSV_FILE_PATH}")

Successfully loaded 500 repositories and sampled 20 of them.


,name,readme,description
83,rdl-standard,"# Risk Data Library Standard\n\nThe Risk Data Library Standard is a data model for describing Hazard, Exposure, \nVulnerability and Loss data.\n\nThe model describes the common core metadata that applies to all risk datasets, \nas well as standardised metadata that applies to Hazard, Exposure, Vulnerability and Loss \ndata.\n\nThis repository is used to coordinate the development of this data model. It will \nbe used to:\n\n* publish working and released drafts of the data model specifications\n* coordinate collaboration and discussion around the iterative development of those specifications\n* provide an overview of the current status and roadmap\n\n## Intended audience\n\nThe repository is intended to support the work of those developing and contributing to the \nRisk Data Library specifications.\n\nThis repository is intended to:\n\n* support comments or feedback on the current specifications\n* propose and discuss changes, e.g. in the form of revised wording or additions to the model\n* answer questions about the governance and evolution of the standard\n\nOther more useful resources exist if you have general questions about the scope and goals \nof [the Risk Data Library project](http://riskdatalibrary.org/), or are looking for a more [high-level introduction to \nthe standard and its key concepts](https://docs.riskdatalibrary.org/).\n\n## How to contribute\n\nThe [Contributors guide](CONTRIBUTING.md) covers the different ways in which you can contribute to this project to \nsupport the development and adoption of the Risk Data Library Standard.\n\n## Project governance\n\nRead [the project governance documentation](GOVERNANCE.md) for more detail about our approach to making decisions and \nagreeing changes to the standard.\n\n## Licence\n\nThe published specifications and all working documents in this repository are published under \na [Creative Commons Attribution-ShareAlike 4.0 (CC-BY-SA 4.0)](https://creativecommons.org/licenses/by-sa/4.0/legalcode) licence.\n\nVisit the Creative Commons website for [official translations of the licence text](https://creativecommons.org/licenses/by-sa/4.0/legalcode#languages).\n","The Risk Data Library Standard (RDLS) is an open data standard to make it easier to work with disaster and climate risk data. It provides a common description of the data used and produced in risk assessments, including hazard, exposure, vulnerability, and modelled loss, or impact, data."
235,GeoNature-mobile-webapi,"# GeoNature-mobile-webapi\n\nGeoNature est une application de saisie et de synthèse des observations faune et flore : https://github.com/PnEcrins/GeoNature\n\nPour pouvoir importer les données saisies avec [Geonature-mobile](https://github.com/PnEcrins/GeoNature-mobile) dans la BDD PostgreSQL de GeoNature, cette web-API doit être installée sur le serveur.\n\nLa synchronisation de ces données peut être faite par le réseau (wifi ou 3G) ou en connectant le mobile en USB à un PC connecté à internet. Dans ce cas, une application de synchronisation des données soit être installée sur le PC : https://github.com/PnEcrins/GeoNature-mobile-sync \n\n![GeoNature schema general](https://github.com/PnEcrins/GeoNature/raw/master/docs/images/schema-geonature-environnement.jpg)\n\n## License\n&copy; Makina Corpus / Parc national des Ecrins 2012 - 2017\n",WebAPI (coté serveur) de synchronisation des données produites par GeoNature-mobile
207,portal-brasil,"<div align=""center""><img alt=""logo"" src=""https://raw.githubusercontent.com/plonegovbr/plonegovbr.portal/main/docs/logo.png"" width=""150"" /></div>\n\n<h1 align=""center"">PortalBrasil</h1>\n\nProjeto de desenvolvimento do Portal Brasil\n\n## Instalação\n\nClone este repositório\n\n```bash\ngit clone git@github.com:plonegovbr/portal-brasil.git\n```\n\nInstale as dependências de backend\n\n```bash\nmake install-backend\n```\n\nInstale as dependências de frontend\n\n```bash\nmake install-frontend\n```\n\n## Inicie os

## 3. Define LLM Utility

This is where you'll integrate your language model. The function `get_summary` is a placeholder. **You should replace its content with the logic from your `llm_utils.py` file.**

The function should accept a `prompt` and the `text_to_summarize` and return the generated summary as a string.

## 4. Define Prompts for Engineering

Here you can define all the different prompts you want to test. I've added a few examples to get you started, focusing on different aspects like tone, format, and content.

In [37]:
# 1. Install necessary libraries if you haven't already
# !pip install langchain-google-genai langchain python-dotenv

import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from your .env file
load_dotenv(dotenv_path='../.env')

# Securely get the API key from the environment.
# This prevents the key from being stored in the notebook's code or output.
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's in your .env file.")

# Define the model name (as seen in llm_utils.py)
LLM_MODEL_NAME = "gemini-1.5-flash"

# Initialize the Google Generative AI model
llm = ChatGoogleGenerativeAI(
    model=LLM_MODEL_NAME,
    google_api_key=google_api_key,
)

# --- Example Usage ---
# You can replace these variables with your own data
repo_description = "A Python library for parsing and generating beautiful, human-readable reports from complex data structures."
repo_readme = """
# Reportify v1.2

Reportify is a tool designed to make data reporting simple. It takes JSON or dictionary data and outputs clean, formatted reports in PDF or HTML.

## Features
- Multiple output formats (PDF, HTML)
- Customizable templates
- Easy integration with pandas DataFrames

## Getting Started
`pip install reportify`
"""

def get_summary(prompt: str, repo_description: str, repo_readme:str) -> str:
    """
    Generates a summary for the given text using a specified prompt.
    This is a placeholder and should be replaced with your actual LLM implementation.

    Args:
        prompt: The prompt to use for the summarization task.
        repo_description: ...
        repo_readme: ...

    Returns:
        The generated summary as a string.
    """

    # This is the prompt template copied directly from llm_utils.py
    summary_prompt = PromptTemplate(
        input_variables=["description", "readme"],
        template=prompt,
    )

    # Create the LangChain chain by piping the components together
    summary_chain = summary_prompt | llm | StrOutputParser()


    # Invoke the chain with your repository data
    return summary_chain.invoke({
        "description": repo_description,
        "readme": repo_readme
    })

# Print the final summary
print(generated_summary)


Reportify is a Python library that simplifies data reporting.  It transforms JSON or dictionary data into aesthetically pleasing PDF or HTML reports.  The library offers customizable templates and integrates easily with pandas DataFrames.  Users can generate formatted reports from complex data structures.


In [44]:
# prompts must include "{description}" and "{readme}"


simple_prompt = """
        Please provide a summary of the following GitHub repository based on its description and README.md content.
        If the README.md or description is not in English, please first translate it to English and then generate a summary.
    
        Repository description:
        {description}
    
        README.md content:
        {readme}
    
        Summary:
        """

short_response_prompt = """
        Please provide a very short summary of the purpose following GitHub repository based on its description and README.md content.
        If the README.md or description is not in English, please first translate it to English and then generate a summary.
        Focus on the intent and purpose of the repository. Not the state or orginsation that created it, the programing language, or nature of the licence.

        The summary should be very breif, around ten words, should not be sentances just relevant, expresive words.    
        
        Repository description:
        {description}
    
        README.md content:
        {readme}
    
        Summary:
        """

bullets_prompt = """
        Please provide a summary bullets of the following GitHub repository based on its description and README.md content.
        If the README.md or description is not in English, please first translate it to English and then generate a summary.
        Focus on the intent and purpose of the repository. Not the state or orginsation that created it, the programing language, or nature of the licence.
        Repository description:
        {description}
    
        README.md content:
        {readme}
    
        Summary:
        """

perscriptive = """
        Please provide a summary of the following GitHub repository based on its description and README.md content.
        These will be used for categorisation via embeddings.
        If the README.md or description is not in English, please first translate it to English and then generate a summary.

        The summary should be:
            * concise and in fewer than 3 sentences.
            * focus on what the repository is used for, enables, what the *intent* of it is
        The summary should not:
            * mention the country, state or organisation that the repository is for.
            * include information about the type of license.
            * comment on where more information can be found or what information was not available.
            * mention the programing language used unless intrinsic to its purpose
            * include the name of the repository unless its a standard word or discription needed to summarise
    
        Repository description:
        {description}
    
        README.md content:
        {readme}
    
        Summary:
        """

prompts_to_test = {
    # 'simple': simple_prompt ,
    # 'bullets':bullets_prompt,
    # 'perscriptive': perscriptive,
    'short': short_response_prompt
}

print(f"Defined {len(prompts_to_test)} prompts to test.")

Defined 1 prompts to test.


## 5. Generate and Compare Summaries

This next cell will iterate through each of the sampled repositories and generate a summary for each of the prompts you defined above. The results will be collected into a DataFrame for easy comparison.

In [45]:
results = []

for index, row in sample_df.iterrows():
    print(f"Processing repository: {row['name']}...")
    repo_info = {
        'name': row['name'],
        'description': row['description'],
        'original_summary': row['summary']
    }
    
    
    for prompt_name, prompt_text in prompts_to_test.items():
        # Generate the summary using the placeholder function
        # In a real run, this will call your LLM
        summary = get_summary(prompt_text, repo_info['description'], row['readme'] )
        repo_info[prompt_name] = summary
        
    results.append(repo_info)

print("\nFinished processing all repositories.")

# Create a DataFrame from the results
results_df = pd.DataFrame(results)


Processing repository: rdl-standard...
Processing repository: GeoNature-mobile-webapi...
Processing repository: portal-brasil...
Processing repository: Alouette_ISIS_extract...
Processing repository: quadratic-voting-frontend...
Processing repository: ala-install...
Processing repository: gsoc-2023...
Processing repository: volto-vlibras...
Processing repository: ioos-code-sprint...
Processing repository: ris-backend-service...
Processing repository: openspace-android-sdk...
Processing repository: steuerlotse...
Processing repository: lexml-renderer-pdf...
Processing repository: uswds...
Processing repository: avh-hub...
Processing repository: invitation-manager...
Processing repository: ioos-python-package-skeleton...
Processing repository: medlink...
Processing repository: peacetrack-readme...
Processing repository: restaurant-inspections...

Finished processing all repositories.


## 6. Review the Results

The table below shows the original summary alongside the new summaries generated by each of your prompts. This should make it easy to compare their effectiveness.

In [46]:
# Set display options to show full text content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

# Display the DataFrame as an HTML table for better readability
display(HTML(results_df.to_html()))

,name,description,original_summary,short
0,rdl-standard,"The Risk Data Library Standard (RDLS) is an open data standard to make it easier to work with disaster and climate risk data. It provides a common description of the data used and produced in risk assessments, including hazard, exposure, vulnerability, and modelled loss, or impact, data.","The Risk Data Library Standard (RDLS) is an open data standard facilitating work with disaster and climate risk data. It provides a common framework for describing hazard, exposure, vulnerability, and loss data used in risk assessments. The standard's development is coordinated through this repository, which publishes specifications and supports collaborative improvements. The repository also encourages feedback and discussion on the evolving standard.","Standardizing disaster risk data: hazard, exposure, vulnerability, loss."
1,GeoNature-mobile-webapi,WebAPI (coté serveur) de synchronisation des données produites par GeoNature-mobile,This repository provides a web API for synchronizing data generated by the GeoNature-mobile application. It enables importing data from GeoNature-mobile into a GeoNature PostgreSQL database. Synchronization can occur via network connection or by connecting a mobile device to a computer. The API is designed to work with a separate synchronization application for PC-based syncing.,"Data synchronization, GeoNature mobile, PostgreSQL database."
2,portal-brasil,Ambiente de desenvolvimento do PortalBrasil,This repository provides the development environment for the PortalBrasil website. It enables the installation and running of both backend and frontend servers using provided make commands. The repository includes instructions for managing backend and frontend packages. Developers can easily set up and manage the PortalBrasil website using this environment.,"PortalBrasil development environment, backend, frontend."
3,Alouette_ISIS_extract,🛰️ Ce code sert à extraire les données et les métadonnées des ionogrammes numérisés des satellites Alouette et ISIS | 🛰️ This code is an effort to extract data and metadata from the scanned ionogram images from the Alouette and ISIS satellites.,"This repository provides tools and data for extracting information from scanned ionograms of the Alouette and ISIS satellites. It offers a centralized repository of data and metadata from these satellites, enabling researchers to access and utilize this historical information for further research. The repository includes open-source code for data processing, raw images, and a user-friendly application for data selection, download, and visualization. The data covers over 60 years of ionospheric research.","Satellite ionogram data extraction, research access, data repository."
4,quadratic-voting-frontend,No description,"This repository provides the frontend code for a quadratic voting system developed for the 2019 Taiwan Presidential Hackathon. It includes sample data files (Proposal.json, ProposalPolls.json, User.json, UserAction.json) representing proposals, voting results, users, and user actions. The repository does not contain backend code. The data can be used for research and analysis.","Quadratic voting, frontend, hackathon project, data, public use."
5,ala-install,Ansible playbooks for installing the ALA components,"This repository provides Ansible playbooks for installing ALA components on Ubuntu 16 and later systems. It includes a playbook for setting up an ALA demo. The playbooks are designed to facilitate the deployment of a Living Atlas, and integrate with supporting tools to simplify the process. These tools assist in generating necessary configuration files and managing the Living Atlas portal.","ALA component installation, Ansible playbooks, Ubuntu deployment."
6,gsoc-2023,Google Summer of Code 2023 with the Mayor's Office of New Urban Mechanics: Guidance + Ideas,This repository contains guidance and ideas developed during Google Summer of Code 2023 in collaboration wit